In [2]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [3]:
llm.invoke("How will the weather be in munich today?")

AIMessage(content="I'm sorry, I am not able to provide real-time weather updates. I recommend checking a reliable weather website or app for the most up-to-date information on the weather in Munich today.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 17, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d82b7d53-6826-4d7b-8627-47b657cf832d-0', usage_metadata={'input_tokens': 17, 'output_tokens': 39, 'total_tokens': 56, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
from langchain_core.tools import tool


@tool(parse_docstring=False)
def get_weather(location: str): # Recommended to mention the type of your arguments so ur llm can understand
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."


@tool
def check_seating_availability(location: str, seating_type: str):
    """Call to check seating availability."""
    if location.lower() == "munich" and seating_type.lower() == "outdoor":
        return "Yes, we still have seats available outdoors."
    elif location.lower() == "munich" and seating_type.lower() == "indoor":
        return "Yes, we have indoor seating available."
    else:
        return "Sorry, seating information for this location is unavailable."


tools = [get_weather, check_seating_availability]

In [13]:
llm_with_tools = llm.bind_tools(tools)

In [14]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result
# The content was empty this means we need to use tool calls

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fG4jlmj6Y9PJExyC8KLRJdS1', 'function': {'arguments': '{"location":"munich"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 84, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b464ebba-7fe8-407e-b46d-ac02b025cbe3-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'munich'}, 'id': 'call_fG4jlmj6Y9PJExyC8KLRJdS1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 16, 'total_tokens': 100, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [19]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'munich'},
  'id': 'call_ClOtT6D7C72N6tidd13lFpBG',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'munich', 'seating_type': 'outdoor'},
  'id': 'call_aea630VUvWNV1Zgj6Vjg0EHr',
  'type': 'tool_call'}]

In [16]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ClOtT6D7C72N6tidd13lFpBG', 'function': {'arguments': '{"location": "munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_aea630VUvWNV1Zgj6Vjg0EHr', 'function': {'arguments': '{"location": "munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 92, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-435504f9-d64a-43bb-8740-69d3148e394f-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'munich'}, 'id': 'call_ClOtT6D7C72N6tidd13lFpBG', 'type': 'tool_call'}, {'name': 

In [22]:
result.tool_calls
# result.response_metadata

[{'name': 'get_weather',
  'args': {'location': 'munich'},
  'id': 'call_ClOtT6D7C72N6tidd13lFpBG',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'munich', 'seating_type': 'outdoor'},
  'id': 'call_aea630VUvWNV1Zgj6Vjg0EHr',
  'type': 'tool_call'}]

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? Do you still have seats outdoor available?"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [ ]:
messages

In [ ]:
tool_mapping = {
    "get_weather": get_weather,
    "check_seating_availability": check_seating_availability,
}

In [ ]:
llm_output.tool_calls

In [ ]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [ ]:
messages

In [ ]:
llm_with_tools.invoke(messages)